# Naivní Bayesův klasifikátor - klasifikace textů

V tomto notebooku se budeme zabývat problémem klasifikace pomocí Naivního Bayese. Speciálně se budeme soustředit na klasifikaci textů.

Základem pro tento dokument je tutorial ze scikit-learn zaměřený na analýzu textů [zde](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).

In [1]:
import pandas as pd
import numpy as np

from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation (so 0.000 is printed as 0.)

## Načtení dat

Využijeme data ze zdroje [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/), který obsahuje cca 20000 textů kategorizovaných do 20 skupin.

Pro jednoduchost se zaměříme pouze na dvě kategorie - hokej a auta.

In [2]:
categories = ['rec.sport.hockey', 'rec.autos']
train_data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=1)
test_data = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=1)

## Prozkoumání trénovacích dat

In [3]:
np.unique(train_data.target, return_counts=True)

(array([0, 1]), array([594, 600]))

In [4]:
print('Kategorie:', train_data.target_names)
print('Train data length:', len(train_data.data))

print('Kategorie prvního dokumentu:',train_data.target_names[train_data.target[0]])
print('---------------------------------------------')
print(train_data.data[0])

Kategorie: ['rec.autos', 'rec.sport.hockey']
Train data length: 1194
Kategorie prvního dokumentu: rec.autos
---------------------------------------------
From: rmt6r@faraday.clas.Virginia.EDU (Roy Matthew Thigpen)
Subject: Re: Ad said Nissan Altima best seller?
Organization: University of Virginia
Lines: 35

boyle@cactus.org  writes:
> In article <1qv7mn$dql@menudo.uh.edu> thang@harebell.egr.uh.edu (Chin-Heng  Thang) writes:
> >	Recently, I saw an ad for the altima which says that it is the  
> >best seller for the past 6 months, is that true? 
> >
> 
> I too was puzzled by this obvious untruth. What I think is going on is that
> Nissan claims that the Altima is "the best selling new car namelplate in
> the US" (I think I have this near verbatim). Lee Iaccoca's statistics
> dept. would have been proud of that sentence. What they mean, I think, is
> that of all "totally new models", i.e. cars never sold before in any
> form, the Altima is the best seller, thereby eliminating Accord, Tau

## Transformace do bag-of-words reprezentace

Použijeme CountVectorizer ze [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html?highlight=countvectorizer#sklearn.feature_extraction.text.CountVectorizer).

Nejprve ale načteme slovník a část zobrazíme.

In [5]:
with open('vocabulary.txt','r') as f:
    vocab=f.read().splitlines()
print('počet slov:',len(vocab))
vocab[:10]

počet slov: 61188


['archive',
 'name',
 'atheism',
 'resources',
 'alt',
 'last',
 'modified',
 'december',
 'version',
 'atheist']

`CountVectorizer()` reprezentuje každý text pomocí vektoru, jehož složky jsou počty výskytů daného slova ve slovníku. Celkem máme $1194$ řádků odpovídajících jednotlivým textům a $61188$ sloupců odpovídajících slovům ve slovníku.

In [6]:
# Navíc: načtení seznamu častých slov s minimálním významem
# from nltk.corpus import stopwords
# stop_words=stopwords.words('english')
stop_words=[]

In [7]:
count_vect = CountVectorizer(vocabulary = vocab, stop_words=stop_words)
X_train_counts = count_vect.fit_transform(train_data.data)
print('Bag of words shape', X_train_counts.shape)
print('Bag of words type', type(X_train_counts))

Bag of words shape (1194, 61188)
Bag of words type <class 'scipy.sparse.csr.csr_matrix'>


Prvních 10 řádků a 20 sloupců matice vypadá jako:

In [8]:
X_train_counts[:10,:20].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0]])

Výstupem je scipy.sparse matrix.

Můžeme se ještě podívat na slova ve slovníku spolu s počty jejich výskytu v prvním textu.

In [9]:
print(type(count_vect.vocabulary_))
print(len(count_vect.vocabulary_))

print({vocab[i]:X_train_counts[0,i] for i in range(20)})

<class 'dict'>
61188
{'archive': 0, 'name': 0, 'atheism': 0, 'resources': 0, 'alt': 0, 'last': 0, 'modified': 0, 'december': 0, 'version': 0, 'atheist': 0, 'addresses': 0, 'of': 6, 'organizations': 0, 'usa': 0, 'freedom': 0, 'from': 2, 'religion': 0, 'foundation': 0, 'darwin': 0, 'fish': 0}


## Aplikace Naivního Bayese s Bernoulliho rozdělením

* Reprezentujeme dokument pomocí indikátorů výskytů slov ze slovníku `vocab`
* Natrénujeme Naivního Bayese s Bernoulliho rozdělením příznaků

In [10]:
# převedeme texty na indikátory výskytu slov
ind_vect = CountVectorizer(binary = True, vocabulary = vocab, stop_words=stop_words)
X_train_indicators = ind_vect.fit_transform(train_data.data)

# zobrazíme výsledek
display(X_train_indicators[:10,:20].toarray())

# natrénujeme klasifikátor
clfi = BernoulliNB().fit(X_train_indicators, train_data.target)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]])

* Otestujeme kvalitu predikce na ručně určených dokumentech
* Odhadneme přesnost (acccuracy) predikce s využitím testovací množiny test_data

In [11]:
# Vlastní dokumenty pro testování
docs_new = ["Lets play hockey.", "I don't like their seats"]

# alternativně možno zkusit
# docs_new = ["Lets play hockey.", "I don't like their game"]

# výpočet indikátorů pomocí natrénovaného CountVectorizer v objektu ind_vect
X_new_indicators = ind_vect.transform(docs_new)

# predikce pro zadané dokumenty
predicted = clfi.predict(X_new_indicators)
probabs = clfi.predict_proba(X_new_indicators)

# print results
for doc, category, probab in zip(docs_new, predicted, probabs):
    print('%r => %s' % (doc, train_data.target_names[category]))
    print(probab)

'Lets play hockey.' => rec.sport.hockey
[0.13849 0.86151]
"I don't like their seats" => rec.autos
[0.99945 0.00055]


In [12]:
# Odhad přesnosti predikce na testovací množině

# výpočet indikátorů pomocí natrénovaného CountVectorizer v objektu ind_vect
X_new_indicators = ind_vect.transform(test_data.data)

# predikce na testovacích datech
predicted = clfi.predict(X_new_indicators)

print('Odhad přesnosti:', clfi.score(X_new_indicators, test_data.target))

Odhad přesnosti: 0.9911949685534591


## Aplikace Naivního Bayese s Multinomiálním rozdělením

* Reprezentujeme dokument pomocí počtů výskytů slov ze slovníku `vocab` - tj. bag-of-words reprezentace
* Natrénujeme multinomického naivního Bayese

In [13]:
# ještě jednou určíme počty slov v jednotlivých textech
count_vect = CountVectorizer(vocabulary = vocab, stop_words=stop_words)
X_train_counts = count_vect.fit_transform(train_data.data)

# natrénujeme klasifikátor
clf = MultinomialNB().fit(X_train_counts, train_data.target)

* Otestujeme kvalitu predikce na ručně určených dokumentech
* Odhadneme přesnost (acccuracy) predikce s využitím testovací množiny test_data

In [14]:
# výpočet četností pomocí natrénovaného CountVectorizer v objektu count_vect
X_new_counts = count_vect.transform(docs_new)

# predikce pro zadané dokumenty
predicted = clf.predict(X_new_counts)
probabs = clf.predict_proba(X_new_counts)

# print results
for doc, category, probab in zip(docs_new, predicted, probabs):
    print('%r => %s' % (doc, train_data.target_names[category]))
    print(probab)

'Lets play hockey.' => rec.sport.hockey
[0.00006 0.99994]
"I don't like their seats" => rec.autos
[0.63915 0.36085]


In [15]:
# Odhad přesnosti predikce na testovací množině

# výpočet četností pomocí natrénovaného CountVectorizer v objektu count_vect
X_new_counts = count_vect.transform(test_data.data)

# predikce na testovacích datech
predicted = clf.predict(X_new_counts)

print('Odhad přesnosti:', clf.score(X_new_counts, test_data.target))

Odhad přesnosti: 0.9962264150943396
